# Task 3

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import OneHotEncoder

from itertools import permutations 

In [2]:
test_data = pd.read_csv('test.csv') #These are just features names (48000)
train_data = pd.read_csv('train.csv') #Says if something is active or not (112000)

num_train_data = len(train_data)

In [3]:
train_feature = np.array(train_data['Sequence'])
train_output = np.array(train_data['Active'])
test_feature = np.array(test_data['Sequence'])

train_feature_splitted = []
test_feature_splitted = []

In [4]:
for i, feature in enumerate(train_feature):
    train_feature_splitted.append([])
    for j, s in enumerate(list(feature)):
        train_feature_splitted[i].append(ord(s)*(j+1))

train_feature_splitted = np.array(train_feature_splitted)

for i, feature in enumerate(test_feature):
    test_feature_splitted.append([])
    for j, s in enumerate(list(feature)):
        test_feature_splitted[i].append(ord(s)*(j+1))

test_feature_splitted = np.array(test_feature_splitted)

In [5]:
for i, j in permutations([0,1,2,3], 2):
    train_feature_mult = np.array([(train_feature_splitted[:,i]+2000)*train_feature_splitted[:,j]]).T
    train_feature_mult = train_feature_mult/np.max(train_feature_mult)
    train_feature_splitted = np.concatenate((train_feature_splitted, train_feature_mult), axis=1)
    
for i, j in permutations([0,1,2,3], 2):
    test_feature_mult = np.array([(test_feature_splitted[:,i]+2000)*test_feature_splitted[:,j]]).T
    test_feature_mult = test_feature_mult/np.max(test_feature_mult)
    test_feature_splitted = np.concatenate((test_feature_splitted, test_feature_mult), axis=1)

In [6]:
train_feature_splitted = np.array(train_feature_splitted, dtype=float)
test_feature_splitted = np.array(test_feature_splitted, dtype=float)

ohe = OneHotEncoder(handle_unknown='ignore')

train_feature_ohe = ohe.fit_transform(train_feature_splitted)
test_feature_ohe = ohe.transform(test_feature_splitted)

print(np.shape(train_feature_ohe))
print(np.shape(test_feature_ohe))

(112000, 4865)
(48000, 4865)


In [7]:
score_func = make_scorer(f1_score)

In [9]:
%%time
kf = KFold(n_splits=3, shuffle=False, random_state=None)
model = MLPClassifier(hidden_layer_sizes=8, activation='tanh'
                      , solver='lbfgs', max_iter=2000, tol=1e-4, alpha=1e-1)

for train_idx, test_idx in kf.split(train_feature_splitted):
    
    model.fit(train_feature_ohe[train_idx], train_output[train_idx])
    
    prediction = model.predict(train_feature_ohe[test_idx])
    
    print(f1_score(train_output[test_idx], prediction))

0.8643880926130099
0.8591499824376537
0.8667621776504298
CPU times: user 34.3 s, sys: 53.5 s, total: 1min 27s
Wall time: 13.5 s


In [ ]:
# Train whole data
model.fit(train_feature_ohe, train_output)

In [16]:
# Make prediction
prediction = model.predict(test_feature_ohe)

(48000,)


In [26]:
pd.DataFrame(prediction.T).to_csv('prediction.csv', index=False, header=None)

In [27]:
pred_data = pd.read_csv('prediction.csv', header=None)